# Práctica 2 - Ejercicios 1 y 2

Resolución de los dos primeros ejercicios de `practica2.md` dentro de este notebook.


## Preparación

Se importan las librerías necesarias y se deja una carga robusta de PyG por si no está disponible en el entorno.


In [1]:
import time
from itertools import combinations

import numpy as np
import networkx as nx

try:
    from torch_geometric.datasets import TUDataset
    from torch_geometric.utils import to_networkx

    PYG_AVAILABLE = True
except Exception as e:
    PYG_AVAILABLE = False
    PYG_IMPORT_ERROR = e


## Exercise 1: Brute Force Baseline (Part 0)

1. Implementar `is_clique()` y `brute_force_max_clique()`
2. Probar en grafos sintéticos pequeños (4-8 nodos)
3. Aplicar a los primeros 20 grafos de IMDB (saltando >25 nodos)
4. Reportar tamaños de clique y tiempos de cómputo


In [2]:
def is_clique(nodes, A):
    """Check if a node subset is a clique in adjacency matrix A."""
    for i, j in combinations(nodes, 2):
        if A[i, j] == 0:
            return False
    return True


def brute_force_max_clique(A):
    """Find a maximum clique by exhaustive search."""
    n = A.shape[0]
    all_nodes = list(range(n))

    for k in range(n, 0, -1):
        for subset in combinations(all_nodes, k):
            if is_clique(subset, A):
                return list(subset)

    return []


In [3]:
def adjacency_from_edges(n, edges):
    A = np.zeros((n, n), dtype=float)
    for u, v in edges:
        A[u, v] = 1.0
        A[v, u] = 1.0
    return A


synthetic_graphs = {
    "G4_triangle_plus_tail": adjacency_from_edges(
        4,
        [(0, 1), (1, 2), (0, 2), (2, 3)],
    ),
    "G5_square_with_diagonal": adjacency_from_edges(
        5,
        [(0, 1), (1, 2), (2, 3), (3, 0), (0, 2), (3, 4)],
    ),
    "G6_with_K4": adjacency_from_edges(
        6,
        [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3), (4, 5), (3, 4)],
    ),
    "G7_with_K4": adjacency_from_edges(
        7,
        [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3), (4, 5), (5, 6)],
    ),
    "G8_with_K5": adjacency_from_edges(
        8,
        [
            (0, 1), (0, 2), (0, 3), (0, 4),
            (1, 2), (1, 3), (1, 4),
            (2, 3), (2, 4),
            (3, 4),
            (5, 6), (6, 7),
        ],
    ),
}

print("Pruebas en grafos sintéticos (4-8 nodos):")
synthetic_results = []
for name, A in synthetic_graphs.items():
    t0 = time.time()
    clique = brute_force_max_clique(A)
    elapsed = time.time() - t0
    synthetic_results.append((name, len(clique), clique, elapsed))
    print(f"- {name}: clique_size={len(clique)}, clique={clique}, time={elapsed:.6f}s")


Pruebas en grafos sintéticos (4-8 nodos):
- G4_triangle_plus_tail: clique_size=3, clique=[0, 1, 2], time=0.000368s
- G5_square_with_diagonal: clique_size=3, clique=[0, 1, 2], time=0.000011s
- G6_with_K4: clique_size=4, clique=[0, 1, 2, 3], time=0.000009s
- G7_with_K4: clique_size=4, clique=[0, 1, 2, 3], time=0.000023s
- G8_with_K5: clique_size=5, clique=[0, 1, 2, 3, 4], time=0.000033s


In [4]:
def evaluate_brute_force_on_dataset(dataset, max_graphs=20, max_nodes=25):
    """Evaluate brute force clique search on small graphs from a PyG dataset."""
    results = []

    for i in range(min(max_graphs, len(dataset))):
        data = dataset[i]

        if data.num_nodes > max_nodes:
            print(f"Graph {i}: skipped ({data.num_nodes} nodes > {max_nodes})")
            continue

        G = to_networkx(data, to_undirected=True)
        A = nx.to_numpy_array(G, nodelist=sorted(G.nodes()), dtype=float)

        t0 = time.time()
        clique = brute_force_max_clique(A)
        elapsed = time.time() - t0

        results.append(
            {
                "graph_idx": i,
                "num_nodes": data.num_nodes,
                "clique_size": len(clique),
                "clique": clique,
                "time_s": elapsed,
            }
        )

        print(
            f"Graph {i}: nodes={data.num_nodes}, "
            f"clique_size={len(clique)}, time={elapsed:.6f}s"
        )

    if results:
        sizes = [r["clique_size"] for r in results]
        times = [r["time_s"] for r in results]
        print("\nResumen IMDB (subconjunto evaluado):")
        print(f"- Grafos evaluados: {len(results)}")
        print(f"- Tamaño medio de clique: {np.mean(sizes):.3f}")
        print(f"- Tiempo medio: {np.mean(times):.6f}s")
    else:
        print("No se pudo evaluar ningún grafo con las restricciones dadas.")

    return results


imdb_results = []
if PYG_AVAILABLE:
    try:
        imdb_dataset = TUDataset(root="./data", name="IMDB-BINARY")
        print(f"IMDB-BINARY cargado: {len(imdb_dataset)} grafos")
        imdb_results = evaluate_brute_force_on_dataset(
            imdb_dataset,
            max_graphs=20,
            max_nodes=25,
        )
    except Exception as e:
        print(f"No se pudo cargar/evaluar IMDB-BINARY: {e}")
else:
    print("torch_geometric no está disponible en este entorno.")
    print(f"Detalle: {PYG_IMPORT_ERROR}")


No se pudo cargar/evaluar IMDB-BINARY: Cannot connect to host www.chrsmrrs.com:443 ssl:True [SSLCertVerificationError: (1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')]


## Exercise 2: Motzkin-Straus Verification (Part 1)

1. Implementar `motzkin_straus_objective()` y `regularized_objective()`
2. Verificar el teorema en 3 grafos de prueba
3. Mostrar solución espuria en el grafo cherry
4. Mostrar cómo la regularización corrige ese problema


In [5]:
def motzkin_straus_objective(x, A):
    """Compute f(x) = x^T A x."""
    x = np.asarray(x, dtype=float)
    return float(x @ A @ x)


def regularized_objective(x, A):
    """Compute f_hat(x) = x^T A x + 0.5 * x^T x."""
    x = np.asarray(x, dtype=float)
    return float((x @ A @ x) + 0.5 * (x @ x))


In [6]:
def simplex_point_from_clique(clique, n):
    x = np.zeros(n, dtype=float)
    if len(clique) > 0:
        x[clique] = 1.0 / len(clique)
    return x


def verify_theorem_on_graph(A, graph_name):
    clique = brute_force_max_clique(A)
    k = len(clique)
    x_star = simplex_point_from_clique(clique, A.shape[0])

    f_star = motzkin_straus_objective(x_star, A)
    theorem_value = 1.0 - (1.0 / k)

    print(f"[{graph_name}] clique={clique}, k={k}")
    print(
        f"  f(x*)={f_star:.6f}, "
        f"1-1/k={theorem_value:.6f}, "
        f"match={np.isclose(f_star, theorem_value)}"
    )

    return {
        "graph": graph_name,
        "clique": clique,
        "k": k,
        "f_x_star": f_star,
        "theorem_value": theorem_value,
        "match": bool(np.isclose(f_star, theorem_value)),
    }


def verify_motzkin_straus():
    # Test 1: Triangle K3 (max clique size 3)
    A1 = np.array(
        [
            [0, 1, 1],
            [1, 0, 1],
            [1, 1, 0],
        ],
        dtype=float,
    )

    # Test 2: Square with diagonal (max clique size 3)
    A2 = np.array(
        [
            [0, 1, 1, 1],
            [1, 0, 1, 0],
            [1, 1, 0, 1],
            [1, 0, 1, 0],
        ],
        dtype=float,
    )

    # Test 3: Cherry graph (max clique size 2)
    A3 = np.array(
        [
            [0, 0, 1],
            [0, 0, 1],
            [1, 1, 0],
        ],
        dtype=float,
    )

    print("Verificación del teorema Motzkin-Straus:")
    results = [
        verify_theorem_on_graph(A1, "Triangle K3"),
        verify_theorem_on_graph(A2, "Square + diagonal"),
        verify_theorem_on_graph(A3, "Cherry"),
    ]

    print("\nDemostración de solución espuria en cherry:")
    x_true = np.array([0.5, 0.0, 0.5])
    x_spurious = np.array([0.25, 0.25, 0.5])

    f_true = motzkin_straus_objective(x_true, A3)
    f_spurious = motzkin_straus_objective(x_spurious, A3)

    fhat_true = regularized_objective(x_true, A3)
    fhat_spurious = regularized_objective(x_spurious, A3)

    print(f"  f(x_true)={f_true:.6f}")
    print(f"  f(x_spurious)={f_spurious:.6f}")
    print(f"  Igualdad sin regularizar: {np.isclose(f_true, f_spurious)}")

    print(f"  f_hat(x_true)={fhat_true:.6f}")
    print(f"  f_hat(x_spurious)={fhat_spurious:.6f}")
    print(f"  Regularización favorece la clique real: {fhat_true > fhat_spurious}")

    return {
        "theorem_checks": results,
        "cherry": {
            "f_true": f_true,
            "f_spurious": f_spurious,
            "fhat_true": fhat_true,
            "fhat_spurious": fhat_spurious,
        },
    }


motzkin_results = verify_motzkin_straus()


Verificación del teorema Motzkin-Straus:
[Triangle K3] clique=[0, 1, 2], k=3
  f(x*)=0.666667, 1-1/k=0.666667, match=True
[Square + diagonal] clique=[0, 1, 2], k=3
  f(x*)=0.666667, 1-1/k=0.666667, match=True
[Cherry] clique=[0, 2], k=2
  f(x*)=0.500000, 1-1/k=0.500000, match=True

Demostración de solución espuria en cherry:
  f(x_true)=0.500000
  f(x_spurious)=0.500000
  Igualdad sin regularizar: True
  f_hat(x_true)=0.750000
  f_hat(x_spurious)=0.687500
  Regularización favorece la clique real: True


## Resultado

Este notebook contiene únicamente la resolución de los ejercicios 1 y 2 solicitados.
